In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard imports
import os
import torch
from tqdm import tqdm
import plotly.express as px

# Imports for displaying vis in Colab / notebook
import webbrowser
import http.server
import socketserver
import threading
PORT = 8000

torch.set_grad_enabled(False)

In [3]:
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"


print(f"Device: {device}")

Device: mps


In [4]:
from datasets import load_dataset  
from transformer_lens import HookedTransformer
from sae_lens import SAE

model = HookedTransformer.from_pretrained("pythia-70m-deduped", device = device)

/Users/adamkarvonen/SAE_bench_template/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/adamkarvonen/SAE_bench_template/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model pythia-70m-deduped into HookedTransformer


In [5]:
# the cfg dict is returned alongside the SAE since it may contain useful information for analysing the SAE (eg: instantiating an activation store)
# Note that this is not the same as the SAEs config dict, rather it is whatever was in the HF repo, from which we can extract the SAE config dict
# We also return the feature sparsities which are stored in HF for convenience. 
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "sae_bench_pythia70m_sweep_topk_ctx128_0730",
    sae_id = "blocks.4.hook_resid_post__trainer_10",
    device = device
)
sae = sae.to(device=device)

print(sae.cfg)

SAEConfig(architecture='standard', d_in=512, d_sae=16384, activation_fn_str='topk', apply_b_dec_to_input=True, finetuning_scaling_factor=False, context_size=128, model_name='pythia-70m-deduped', hook_name='blocks.4.hook_resid_post', hook_layer=4, hook_head_index=None, prepend_bos=True, dataset_path='monology/pile-uncopyrighted', dataset_trust_remote_code=False, normalize_activations='none', dtype='float32', device='mps', sae_lens_training_version=None, activation_fn_kwargs={'k': 80}, neuronpedia_id=None, model_from_pretrained_kwargs={})
